In [ ]:
library(ggplot2)
library(gdata)
library(stringr)
library(RColorBrewer)
options(sqldf.driver = "SQLite") # as per FAQ #7 force SQLite
options(gsubfn.engine = "R")
library(sqldf)
library(scales)
library(reshape2)
library(Hmisc)
library(readr)
library(tidyr)
library(dplyr)
library(forcats)
library(gridExtra)

#mass = read_csv("results/20181218_151403/20181218_151403.csv")

mass = read_csv("results/latest/all.csv")
describe(mass$Number_Partitions)

In [ ]:
summary(mass)

In [ ]:

#mass_filtered <-mass %>% filter(Bytes_per_Message<400000 & Number_Points_per_Message ==5000)

mass_long = mass %>% drop_na(Time) 

mass_long = mass %>% drop_na() %>%
  mutate(Bytes_per_sec = Records_per_sec*Bytes_per_Message)%>%
  mutate(MBytes_per_sec = Bytes_per_sec/1000000,
         Broker_Scenario = paste(Number_Brokers, "Broker(s)"))%>%
  mutate(Scenario = paste(Application, "-", Number_Points_per_Message, sep=""))%>%
  #filter(Number_Points_per_Message!=10000 & Number_Points_per_Message!=20000)%>%
  select(c(Application, Scenario, Broker_Scenario, Number_Points_per_Message, Number_Processes, 
           Number_Brokers, Number_Partitions, Number_Messages, Bytes_per_Message, Time, Points_per_sec,Records_per_sec,
           Bytes_per_sec,MBytes_per_sec)) %>%
  gather(type, value, c(Records_per_sec, MBytes_per_sec, Time)) %>%
  mutate(type = fct_recode(as.factor(type), 
                           "Messages/sec" = "Records_per_sec" , 
                           "MB/sec" = "MBytes_per_sec"  )) # %>% 
 # mutate(Application = fct_recode(as.factor(Application), 
 #                                 "KMeans-Random" = "kmeans" , 
 #                                 "Lightsource-static" = "light",
 #                                 "KMeans-Static" = "kmeansstatic"
 #                                 )) 



pp <- ggplot(mass_long, aes(x=Number_Processes, y=value, color=as.factor(Bytes_per_Message), fill=Application)) +
  geom_line(stat="summary", fun.y="mean",  size=1.1) + geom_point(stat="summary", fun.y="mean", size=1.7, show.legend = F) +
  stat_summary(fun.data = "mean_sdl", geom = "errorbar", fun.args = list(mult=1), width=.2) + 
  #geom_bar(stat="summary", fun.y="mean",  size=1.05, position = position_dodge()) + #geom_point(stat="summary", fun.y="mean", size=1.7, show.legend = F) +
  facet_grid(type~Number_Partitions,  scales = "free_y", space = "fixed")+
  #geom_errorbar(mapping=aes(ymin=mean-stddev, ymax=mean+stddev), width=0.2, size=0.6, position = position_dodge(width = 0.90), color="black") +
  scale_fill_brewer(palette="Paired")+ scale_color_brewer(palette="Paired")+ 
  theme_bw(base_size = 20) + theme(legend.position = "bottom", plot.margin=grid::unit(c(0,0,0,-6), "mm")) + 
  #theme(axis.text.x = element_text(angle=45, hjust=1)) +
  scale_x_continuous(trans=log2_trans(), breaks=c(1,2,4,8,16,32,64,128))+ 
  #scale_y_continuous(trans=log2_trans())+   
  #scale_y_log10(breaks=c(1,10,100,200, 500))+   
  xlab("Number Producer Cores") + 
  ylab("") +
  guides(fill=guide_legend(reverse=FALSE, ncol=5, keywidth=1)) +
  labs(fill = "Mini App Streaming Source:")
pp